In [1]:
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np

# Constants
C5, C6, C7, C8 = 1.0, 1.0, 1.0, 1.0
sigma = 0.2

# Helper function to compute vicinal points and weights
def compute_vicinal_points_and_weights(y, eps):
    y_eps = y + eps
    weights = K.exp(-K.square(eps) / (2 * K.square(sigma)))
    return y_eps, weights

# Indicator function
def indicator_function(y, y_tilde, kappa):
    return K.cast(K.abs(y - y_tilde) <= kappa, tf.float32)


In [2]:
def hvdl_loss(y_real, y_fake, D_real, D_fake, kappa):
    N_r = tf.shape(y_real)[0]
    N_g = tf.shape(y_fake)[0]
    
    eps_r = tf.random.normal(tf.shape(y_real), mean=0.0, stddev=sigma)
    eps_g = tf.random.normal(tf.shape(y_fake), mean=0.0, stddev=sigma)
    
    y_real_eps, w_real = compute_vicinal_points_and_weights(y_real, eps_r)
    y_fake_eps, w_fake = compute_vicinal_points_and_weights(y_fake, eps_g)
    
    term1 = K.mean(indicator_function(y_real, y_real_eps, kappa) * K.log(D_real + K.epsilon()))
    term2 = K.mean(indicator_function(y_fake, y_fake_eps, kappa) * K.log(1 - D_fake + K.epsilon()))
    
    loss = -C5/N_r * term1 - C6/N_g * term2
    return loss


In [3]:
def svdl_loss(y_real, y_fake, D_real, D_fake):
    N_r = tf.shape(y_real)[0]
    N_g = tf.shape(y_fake)[0]
    
    eps_r = tf.random.normal(tf.shape(y_real), mean=0.0, stddev=sigma)
    eps_g = tf.random.normal(tf.shape(y_fake), mean=0.0, stddev=sigma)
    
    y_real_eps, w_real = compute_vicinal_points_and_weights(y_real, eps_r)
    y_fake_eps, w_fake = compute_vicinal_points_and_weights(y_fake, eps_g)
    
    term1 = K.mean(w_real * K.log(D_real + K.epsilon()))
    term2 = K.mean(w_fake * K.log(1 - D_fake + K.epsilon()))
    
    loss = -C7/N_r * term1 - C8/N_g * term2
    return loss


In [5]:
import tensorflow as tf

# Placeholder for real and fake labels
y_real = tf.Variable(tf.zeros(shape=(None, 1)), dtype=tf.float32)
y_fake = tf.Variable(tf.zeros(shape=(None, 1)), dtype=tf.float32)

# Real and fake predictions from discriminator
D_real = tf.Variable(tf.zeros(shape=(None, 1)), dtype=tf.float32)
D_fake = tf.Variable(tf.zeros(shape=(None, 1)), dtype=tf.float32)

# Placeholder for epsilon
epsilon_r = tf.Variable(tf.random.normal(shape=(None, 1), mean=0.0, stddev=0.2), dtype=tf.float32)
epsilon_g = tf.Variable(tf.random.normal(shape=(None, 1), mean=0.0, stddev=0.2), dtype=tf.float32)

# Vicinal points
y_r_eps, w_r_eps = compute_vicinal_points_and_weights(y_real, epsilon_r)
y_g_eps, w_g_eps = compute_vicinal_points_and_weights(y_fake, epsilon_g)

# HVDL Loss
HVDL = C5 * tf.reduce_mean(indicator_function(y_real, y_r_eps, kappa) * w_r_eps * tf.math.log(D_real + 1e-10)) - \
       C6 * tf.reduce_mean(indicator_function(y_fake, y_g_eps, kappa) * w_g_eps * tf.math.log(1.0 - D_fake + 1e-10))

# SVDL Loss
SVDL = C7 * tf.reduce_mean(w_r_eps * tf.math.log(D_real + 1e-10)) - \
       C8 * tf.reduce_mean(w_g_eps * tf.math.log(1.0 - D_fake + 1e-10))

# Generator Loss
G_loss = tf.reduce_mean(tf.math.log(D_fake + 1e-10))


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.